# Song Recomender Model

### Notes:
1. numerical variables. (can do research and find which features are relevant for model)
2. scale
3. train k means (+pickle)

- create model

user input
- search for track id
- get audio features
- scale
- predict cluster
- recommend random song cluster

X
X-noid = X.drop("ID")

kmeans.labels
x["cluster"] = kmeans.labels



Tip. Go through whole process first and get quick and dirty model.



to train model only use numerical values
can potentially drop some numerical features to improve model(recommended) (research)


### Which feature to use
For a K-means model that recommends songs based on user input, focusing on Spotify audio features that capture the essence of a song's mood, energy, and danceability can be particularly effective. Consider these features:

Danceability: Reflects the suitability of a song for dancing, based on tempo, rhythm stability, beat strength, and overall regularity.
Energy: Measures intensity and activity, capturing the dynamic feel of a song.
Valence: Indicates the musical positiveness conveyed by a track, which can help in understanding the emotional context.
Tempo: The speed or pace of a song, which is fundamental in matching songs with a similar vibe.
Acousticness: Helps in distinguishing between acoustic and more electronic/synthetic music.
Instrumentalness: Useful for identifying songs with a focus on instrumentation, which might be preferred by users interested in instrumental tracks.
Liveness: Could be considered if the presence of live audience sounds or a "live" feel is important for similarity.
Speechiness: Can be useful to filter out tracks with more spoken words, distinguishing between music tracks and those with significant vocal content like podcasts or audiobooks.
These features collectively can capture the essence of what a user might enjoy in a song, beyond just genre or artist. By clustering songs using these features, your K-means model can identify and recommend songs that share similar characteristics with the user's input song, potentially leading to more personalized and satisfying recommendations.

Remember, the choice of features might require iteration; you might start with these and adjust based on the performance of your recommendation system and user feedback.

## Importing libraries

In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn import datasets # sklearn comes with some toy datasets to practice
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score

## Loading the dataset

In [10]:
song_database_df = pd.read_csv('../data/full_af.csv',index_col=0)
song_database_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.382,0.451,7,-6.606,0,0.0293,0.55700,0.000034,0.2650,0.506,150.803,audio_features,6yzJiHzsscf0n88xb9aNT4,spotify:track:6yzJiHzsscf0n88xb9aNT4,https://api.spotify.com/v1/tracks/6yzJiHzsscf0...,https://api.spotify.com/v1/audio-analysis/6yzJ...,267540,4
1,0.742,0.375,7,-13.829,1,0.0482,0.39700,0.901000,0.0996,0.105,102.023,audio_features,5zdXRoKLZ54VBdbdn7yhz4,spotify:track:5zdXRoKLZ54VBdbdn7yhz4,https://api.spotify.com/v1/tracks/5zdXRoKLZ54V...,https://api.spotify.com/v1/audio-analysis/5zdX...,342163,4
2,0.585,0.552,0,-10.613,1,0.0353,0.19400,0.017100,0.0735,0.496,100.089,audio_features,5qvCKAyjgHnpR5c63OedCY,spotify:track:5qvCKAyjgHnpR5c63OedCY,https://api.spotify.com/v1/tracks/5qvCKAyjgHnp...,https://api.spotify.com/v1/audio-analysis/5qvC...,267813,4
3,0.405,0.535,6,-7.504,0,0.0531,0.05310,0.106000,0.0696,0.637,97.698,audio_features,2vByA9JgfMU2bPjUqmxCdv,spotify:track:2vByA9JgfMU2bPjUqmxCdv,https://api.spotify.com/v1/tracks/2vByA9JgfMU2...,https://api.spotify.com/v1/audio-analysis/2vBy...,210800,4
4,0.586,0.526,0,-6.928,1,0.0564,0.55500,0.000000,0.2340,0.456,107.153,audio_features,0pPQwIjwqduvLfZORYYEaI,spotify:track:0pPQwIjwqduvLfZORYYEaI,https://api.spotify.com/v1/tracks/0pPQwIjwqduv...,https://api.spotify.com/v1/audio-analysis/0pPQ...,161267,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19368,0.876,0.786,5,-3.515,1,0.0787,0.20700,0.812000,0.0724,0.808,154.024,audio_features,7Ca06cMSgTwcz2JfxvAD3E,spotify:track:7Ca06cMSgTwcz2JfxvAD3E,https://api.spotify.com/v1/tracks/7Ca06cMSgTwc...,https://api.spotify.com/v1/audio-analysis/7Ca0...,204560,4
19369,0.423,0.954,7,-3.157,0,0.0470,0.00210,0.000000,0.1770,0.500,144.553,audio_features,3jJZVeExYzVYiV6Y9Fl3DX,spotify:track:3jJZVeExYzVYiV6Y9Fl3DX,https://api.spotify.com/v1/tracks/3jJZVeExYzVY...,https://api.spotify.com/v1/audio-analysis/3jJZ...,310347,4
19370,0.687,0.606,7,-5.920,1,0.0262,0.17800,0.000326,0.0830,0.748,80.569,audio_features,0jpk88zk40MjQ63ljrq7V2,spotify:track:0jpk88zk40MjQ63ljrq7V2,https://api.spotify.com/v1/tracks/0jpk88zk40Mj...,https://api.spotify.com/v1/audio-analysis/0jpk...,206307,4
19371,0.711,0.718,6,-5.739,1,0.0380,0.00824,0.002080,0.0732,0.688,117.071,audio_features,0sEm1ld0V8YTCPcjPVfIsc,spotify:track:0sEm1ld0V8YTCPcjPVfIsc,https://api.spotify.com/v1/tracks/0sEm1ld0V8YT...,https://api.spotify.com/v1/audio-analysis/0sEm...,200293,4


In [11]:
#getting only numeric features for model
song_database_numeric_noid_df = song_database_df.select_dtypes(include=['number'])
song_database_numeric_noid_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0.382,0.451,7,-6.606,0,0.0293,0.55700,0.000034,0.2650,0.506,150.803,267540,4
1,0.742,0.375,7,-13.829,1,0.0482,0.39700,0.901000,0.0996,0.105,102.023,342163,4
2,0.585,0.552,0,-10.613,1,0.0353,0.19400,0.017100,0.0735,0.496,100.089,267813,4
3,0.405,0.535,6,-7.504,0,0.0531,0.05310,0.106000,0.0696,0.637,97.698,210800,4
4,0.586,0.526,0,-6.928,1,0.0564,0.55500,0.000000,0.2340,0.456,107.153,161267,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19368,0.876,0.786,5,-3.515,1,0.0787,0.20700,0.812000,0.0724,0.808,154.024,204560,4
19369,0.423,0.954,7,-3.157,0,0.0470,0.00210,0.000000,0.1770,0.500,144.553,310347,4
19370,0.687,0.606,7,-5.920,1,0.0262,0.17800,0.000326,0.0830,0.748,80.569,206307,4
19371,0.711,0.718,6,-5.739,1,0.0380,0.00824,0.002080,0.0732,0.688,117.071,200293,4


In [14]:
#getting df with numeric features plus id
song_database_numeric_id_df = pd.concat([song_database_df["id"],song_database_numeric_noid_df], axis = 1)
song_database_numeric_id_df

,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,6yzJiHzsscf0n88xb9aNT4,0.382,0.451,7,-6.606,0,0.0293,0.55700,0.000034,0.2650,0.506,150.803,267540,4
1,5zdXRoKLZ54VBdbdn7yhz4,0.742,0.375,7,-13.829,1,0.0482,0.39700,0.901000,0.0996,0.105,102.023,342163,4
2,5qvCKAyjgHnpR5c63OedCY,0.585,0.552,0,-10.613,1,0.0353,0.19400,0.017100,0.0735,0.496,100.089,267813,4
3,2vByA9JgfMU2bPjUqmxCdv,0.405,0.535,6,-7.504,0,0.0531,0.05310,0.106000,0.0696,0.637,97.698,210800,4
4,0pPQwIjwqduvLfZORYYEaI,0.586,0.526,0,-6.928,1,0.0564,0.55500,0.000000,0.2340,0.456,107.153,161267,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19368,7Ca06cMSgTwcz2JfxvAD3E,0.876,0.786,5,-3.515,1,0.0787,0.20700,0.812000,0.0724,0.808,154.024,204560,4
19369,3jJZVeExYzVYiV6Y9Fl3DX,0.423,0.954,7,-3.157,0,0.0470,0.00210,0.000000,0.1770,0.500,144.553,310347,4
19370,0jpk88zk40MjQ63ljrq7V2,0.687,0.606,7,-5.920,1,0.0262,0.17800,0.000326,0.0830,0.748,80.569,206307,4
19371,0sEm1ld0V8YTCPcjPVfIsc,0.711,0.718,6,-5.739,1,0.0380,0.00824,0.002080,0.0732,0.688,117.071,200293,4


## Scaling features

In [16]:
X = song_database_numeric_noid_df
X.describe()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
count,19373.000000,19373.000000,19373.000000,19373.000000,19373.000000,19373.000000,19373.000000,19373.000000,19373.000000,19373.000000,19373.000000,1.937300e+04,19373.000000
mean,0.569374,0.619557,5.202498,-8.866614,0.688639,0.064091,0.285062,0.082866,0.192702,0.547490,120.517385,2.357021e+05,3.930625
std,0.155995,0.224611,3.582950,4.068685,0.463062,0.072843,0.295883,0.218065,0.166637,0.253886,27.849324,8.201893e+04,0.342372
min,0.000000,0.000000,0.000000,-60.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.240000e+03,0.000000
25%,0.464000,0.462000,2.000000,-11.045000,0.000000,0.032400,0.031500,0.000000,0.092400,0.343000,100.015000,1.903470e+05,4.000000
50%,0.579000,0.647000,5.000000,-8.176000,1.000000,0.040400,0.166000,0.000088,0.125000,0.553000,119.881000,2.257500e+05,4.000000
75%,0.680000,0.805000,9.000000,-5.954000,1.000000,0.060700,0.491000,0.010100,0.244000,0.761000,136.095000,2.656000e+05,4.000000
max,0.988000,0.999000,11.000000,-0.320000,1.000000,0.963000,0.996000,0.996000,0.997000,0.993000,239.964000,1.967400e+06,5.000000


In [17]:
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns = X.columns)
display(X.head())
print()
display(X_scaled_df.head())

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0.382,0.451,7,-6.606,0,0.0293,0.5570,0.000034,0.2650,0.506,150.803,267540,4
1,0.742,0.375,7,-13.829,1,0.0482,0.3970,0.901000,0.0996,0.105,102.023,342163,4
2,0.585,0.552,0,-10.613,1,0.0353,0.1940,0.017100,0.0735,0.496,100.089,267813,4
3,0.405,0.535,6,-7.504,0,0.0531,0.0531,0.106000,0.0696,0.637,97.698,210800,4
4,0.586,0.526,0,-6.928,1,0.0564,0.5550,0.000000,0.2340,0.456,107.153,161267,4


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,-1.201182,-0.750458,0.501695,0.555627,-1.487180,-0.477637,0.919095,-0.379860,0.433873,-0.163425,1.087509,0.388188,0.202635
1,1.106640,-1.088829,0.501695,-1.219685,0.672414,-0.218167,0.378327,3.751896,-0.558727,-1.742918,-0.664105,1.298038,0.202635
2,0.100173,-0.300781,-1.452053,-0.429237,0.672414,-0.395266,-0.307772,-0.301596,-0.715359,-0.202814,-0.733551,0.391517,0.202635
3,-1.053738,-0.376469,0.222588,0.334912,-1.487180,-0.150897,-0.783986,0.106092,-0.738763,0.352569,-0.819409,-0.303621,0.202635
4,0.106584,-0.416540,-1.452053,0.476484,0.672414,-0.105593,0.912336,-0.380015,0.247835,-0.360369,-0.479894,-0.907559,0.202635


In [18]:
X_scaled_df.describe()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
count,1.937300e+04,1.937300e+04,1.937300e+04,1.937300e+04,1.937300e+04,1.937300e+04,1.937300e+04,1.937300e+04,1.937300e+04,1.937300e+04,1.937300e+04,1.937300e+04,1.937300e+04
mean,3.374280e-16,-8.802470e-17,-1.192001e-18,1.056296e-16,-7.628808e-17,1.760494e-17,-5.574898e-17,-5.868314e-18,-1.320371e-16,4.225186e-16,-1.680722e-16,2.787449e-17,-2.017233e-16
std,1.000026e+00,1.000026e+00,1.000026e+00,1.000026e+00,1.000026e+00,1.000026e+00,1.000026e+00,1.000026e+00,1.000026e+00,1.000026e+00,1.000026e+00,1.000026e+00,1.000026e+00
min,-3.650039e+00,-2.758422e+00,-1.452053e+00,-1.256787e+01,-1.487180e+00,-8.798842e-01,-9.634532e-01,-3.800150e-01,-1.156447e+00,-2.156501e+00,-4.327591e+00,-2.785552e+00,-1.148086e+01
25%,-6.755117e-01,-7.014835e-01,-8.938392e-01,-5.354167e-01,-1.487180e+00,-4.350785e-01,-8.569895e-01,-3.800150e-01,-6.019358e-01,-8.054630e-01,-7.362087e-01,-5.529971e-01,2.026353e-01
50%,6.170937e-02,1.221822e-01,-5.651867e-02,1.697434e-01,6.724136e-01,-3.252499e-01,-4.024064e-01,-3.796100e-01,-4.062964e-01,2.170276e-02,-2.285159e-02,-1.213416e-01,2.026353e-01
75%,7.091818e-01,8.256373e-01,1.059909e+00,7.158799e-01,6.724136e-01,-4.655986e-02,6.960287e-01,-3.336972e-01,3.078474e-01,8.409908e-01,5.593679e-01,3.645344e-01,2.026353e-01
max,2.683652e+00,1.689373e+00,1.618122e+00,2.100638e+00,6.724136e-01,1.234073e+01,2.402828e+00,4.187558e+00,4.826758e+00,1.754812e+00,4.289142e+00,2.111394e+01,3.123509e+00


## Clustering with K-Means

In [26]:
kmeans = KMeans(n_clusters=13, random_state=1234)
kmeans.fit(X_scaled_df)

/Users/jameskenny/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=13, random_state=1234)

In [27]:
labels = kmeans.labels_
labels

array([1, 5, 2, ..., 4, 4, 3], dtype=int32)

In [28]:
np.unique(labels)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12], dtype=int32)

In [29]:
clusters = kmeans.predict(X_scaled_df)
#clusters
pd.Series(clusters).value_counts().sort_index()

0      230
1     3240
2     2540
3      993
4     2733
5     1027
6      102
7      423
8     2595
9     1918
10    1919
11     913
12     740
Name: count, dtype: int64

In [30]:
clusters

array([1, 5, 2, ..., 4, 4, 3], dtype=int32)

In [32]:
#X_df = pd.DataFrame(X)
X["cluster"] = clusters
X

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,cluster
0,0.382,0.451,7,-6.606,0,0.0293,0.55700,0.000034,0.2650,0.506,150.803,267540,4,1
1,0.742,0.375,7,-13.829,1,0.0482,0.39700,0.901000,0.0996,0.105,102.023,342163,4,5
2,0.585,0.552,0,-10.613,1,0.0353,0.19400,0.017100,0.0735,0.496,100.089,267813,4,2
3,0.405,0.535,6,-7.504,0,0.0531,0.05310,0.106000,0.0696,0.637,97.698,210800,4,1
4,0.586,0.526,0,-6.928,1,0.0564,0.55500,0.000000,0.2340,0.456,107.153,161267,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19368,0.876,0.786,5,-3.515,1,0.0787,0.20700,0.812000,0.0724,0.808,154.024,204560,4,5
19369,0.423,0.954,7,-3.157,0,0.0470,0.00210,0.000000,0.1770,0.500,144.553,310347,4,10
19370,0.687,0.606,7,-5.920,1,0.0262,0.17800,0.000326,0.0830,0.748,80.569,206307,4,4
19371,0.711,0.718,6,-5.739,1,0.0380,0.00824,0.002080,0.0732,0.688,117.071,200293,4,4


In [33]:
X[X['cluster'] == 2].sample()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,cluster
4178,0.698,0.635,0,-10.287,1,0.026,0.255,0.000085,0.0696,0.953,103.988,260573,4,2


## Playing with the parameters

## Choosing K

## Silhouette

## Saving with pickle

In [36]:
import pickle

#scaler = StandardScaler()
#model = KMeans()

with open("model/scaler.pickle", "wb") as f:
    pickle.dump(scaler,f)

with open("model/kmeans_4.pickle", "wb") as f:
    pickle.dump(kmeans,f)

In [ ]:
def load(filename = "filename.pickle"): 
    try: 
        with open(filename, "rb") as f: 
            return pickle.load(f) 
        
    except FileNotFoundError: 
        print("File not found!")

In [ ]:
scaler= load(filename="Model/scaler.pickle")

In [ ]:
kmeans_model = load(filename="Model/kmeans_4.pickle")

In [ ]:
kmeans_model

In [ ]:
scaler2 = load("Model/scaler.pickle")